In [1]:
import numpy as np
import sys 
sys.path.append('../../../')
from stateinterpreter.numerical_utils import gaussian_kde
import matplotlib.pyplot as plt
from statistics import mean
from scipy.optimize import minimize
from tqdm import tqdm

In [ ]:
dims = 2
mean_val = np.random.rand(2)*10
cov = np.random.rand(2,2)
cov = cov@cov.T
size = 100000
samples = np.random.multivariate_normal(mean_val, cov, size=size)

In [ ]:
plt.plot(samples[:,0], samples[:,1], 'k.')
plt.plot(mean_val[0], mean_val[1], 'wx')
plt.axis('equal')

In [ ]:
plt.hexbin(samples[:,0], samples[:,1])
plt.plot(mean_val[0], mean_val[1], 'ko')
plt.plot(res[:,0], res[:,1], 'k.')
plt.axis('equal')

In [ ]:
logpdf = True
sizes = np.arange(0, 100000, 5000)[1:]
residuals = []
for size in tqdm(sizes):
    subsample_idxs = np.random.randint(0,len(samples) -1, size=size)
    subsample = samples[subsample_idxs]
    KDE = gaussian_kde(subsample)
    jac = lambda x: -KDE.grad(x, logpdf=logpdf)
    objective = lambda x: -KDE(x, logpdf=logpdf)
    res = minimize(objective, [-2,-1], jac=jac, tol=1e-8)
    residuals.append(res.x)
res = np.asarray(residuals)
res_nrm = np.linalg.norm(res, axis=1)

In [ ]:
plt.plot(sizes, res_nrm, 'k-')
plt.plot(sizes, res_nrm, 'ko')
plt.xlim(sizes[0], sizes[-1])

In [ ]:
plt.plot(sizes, res_nrm, 'k-')
plt.plot(sizes, res_nrm, 'ko')
plt.xlim(sizes[0], sizes[-1])

In [2]:
def test_gaussian_kde(n_centers = 1000, n_dims = 3, n_pts = 1000, logpdf = False):
    """Sample test, will always pass so long as import statement worked."""
    rand_dataset = np.random.rand(n_centers, n_dims)
    rand_points = np.random.rand(n_pts, n_dims)

    #Test 
    KDE = gaussian_kde(rand_dataset)
    if logpdf:
        return KDE.logpdf(rand_points)
    else:
        return KDE(rand_points)

In [ ]:
n_centers_vals = np.arange(1000,30000,1000)
time_log = []
time_std = []
#n_pts = 1
n_centers = 10000
for n_pts in n_centers_vals:
    val = %timeit -o test_gaussian_kde(n_centers = n_centers, n_dims = 3, n_pts = n_pts, logpdf = False)
    time_std.append(mean(val.timings))
    val = %timeit -o test_gaussian_kde(n_centers = n_centers, n_dims = 3, n_pts = n_pts, logpdf = True)
    time_log.append(mean(val.timings))

In [5]:
%timeit -o test_gaussian_kde(n_centers = 30000, n_dims = 2, n_pts = 300, logpdf = True)

134 ms ± 1.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<TimeitResult : 134 ms ± 1.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)>

In [ ]:
plt.plot(n_centers_vals, time_log, 'k-')
plt.plot(n_centers_vals, time_log, 'ko')
plt.plot(n_centers_vals, time_std, 'k--')
plt.plot(n_centers_vals, time_std, 'ko')
plt.xlim(n_centers_vals[0], n_centers_vals[-1])

In [6]:
import numba as nb

In [7]:
nb.__version__

'0.51.2'